These classes form the backbone of Alicat operations through serial commmands

In [1]:
try:
    import serial
except ImportError:
    print("An error occurred while attempting to import the pyserial backend. Please check your installation and try again.")
import time

In [2]:
class Serial_Connection(object):
    """The serial connection object from which other classes inherit port and baud settings as
    well as reading and writing capabilities. Preprocessing for GP firmware outputs is performed
    using the remove_characters function"""
    
    # A dictionary storing open serial ports for comparison with new connections being opened
    open_ports = {}
    
    def __init__(self, port='/dev/ttyUSB0', baud=19200):
        
        self.port, self.baud = port, baud
        
        # Checks for the existence of an identical port to avoid multiple instances
        # creates a new one if none exists
        if port in Serial_Connection.open_ports:
            self.connection = Serial_Connection.open_ports[port]
        else:
            self.connection = serial.Serial(port, baud, timeout=2.0)
            Serial_Connection.open_ports[port] = self.connection
        
        self.open = True
        
    def _test_open(self):
        
        if not self.open:
            raise IOError(f"The connection to Alicats on port {self.port} not open")
    
    
    def _flush(self):
        """Deletes all characters in the read/write buffer to avoid double messages or rewrites"""
        self._test_open()
        
        self.connection.flush()
        self.connection.flushInput()
        self.connection.flushOutput()

    def _close(self):
        """Checks if the instance exists, clears the buffer, closes the connection, and removes
        itself from the dictionary of active ports"""
        if not self.open:
            return
        
        self._flush()
        
        self.connection.close()
        Serial_Connection.open_ports.pop(self.port, None)
        
        self.open = False
        
    def _read(self):
        """Fast read method using byte arrays with a carriage return delimiter between lines.
        ~30x faster than a readline operation by reading individual characters and appending 
        them to the byte array. The byte array is then decoded into a string and returned"""
        self._test_open()
        
        line = bytearray()
        while True:
            c = self.connection.read(1)
            if c:
                line += c
                if line[-1] == ord('\r'):
                    break
            else:
                break
        
        return line.decode('ascii').strip().replace('\x08','')
    
    def _write(self, ID, command, verbose=False):
        """Writes the input command for a device with the stated ID by encoding to ascii
        and sending through the serial connection write command. Reads out the return parsing 
        multiple lines until the buffer is clear."""
        command = str(ID) + str(command) + '\r'
        command = command.encode('ascii')
        self.connection.write(command)
        if verbose:
            response = []
            response.append(self._read())
            while True:
                if response[-1] != '':
                    response.append(self._read())
                else:
                    return response[:-1]
            
        else:
            self._flush()
    

In [88]:
class MassFlowMeter(Serial_Connection):
    """Base laminar dP mass flow object which contains all """
    
    def __init__(self, ID :str ='A', port : str ='/dev/ttyUSB0', baud : int =19200):
        super().__init__(port, baud)
        self.ID, self.port, self.baud = ID, port, baud
        self._fetch_gas_list()
        self._fetch_firmware_version()
        self._data_format()
        self.current_gas = self._write(self.ID,'',True)[-1].split()[-1]
        self.gas_changes = 0
        self.mix_changes = 0
        
    
    def _fetch_gas_list(self):
        self.gas_list = {}
        self.gas_ref = {}
        self.reverse_gas_list = {}
        gases = [i.split() for i in self._write(self.ID, '??G*',True)]
        for i in range(len(gases)):
            self.gas_list[gases[i][2]] = int(gases[i][1][1:])
            self.gas_ref[int(gases[i][1][1:])] = int(gases[i][1][1:])
            self.reverse_gas_list[int(gases[i][1][1:])] = gases[i][2]
            self.gas_ref.update(self.gas_list)
        del gases
        
    
    def _fetch_firmware_version(self):
        self.manufacturer_data = [i.split() for i in self._write(self.ID, '??M*',True)]
        self.firmware_version = self.manufacturer_data[-1][-1]
        if self.firmware_version[:2] != 'GP':
            self.firmware_version, self.firmware_minor = self.firmware_version.split('v',1)
            self.firmware_version = int(self.firmware_version)
        else:
            self.firmware_version, self.firmware_minor = 'GP', '07'
        
        
    def _data_format(self):
        self.data = []
        outputs = [i.split() for i in self._write(self.ID,'??D*',True)]
        for i in range(len(outputs)):
            outputs[i][1] = outputs[i][1][1:]
        if isinstance(self.firmware_version, str) or (isinstance(self.firmware_version,int) and self.firmware_version <= 6):
            # for 6v and older format
            for i in range(2, len(outputs)):
                c = outputs[i]
                if c[-1] == 'na' or '_':
                    del c[-1]
                self.data.append([int(c[1]), c[2], c[-1]])
        
        else:
            # for 7v and newer format
            output2 = []
            for i in range(len(outputs)):
                c = outputs[i]
                for i in range(len(c)):
                    try:
                        c[i] = int(c[i])
                    except ValueError:
                        pass
                seq = 0
                for i in range(len(c)):
                    if isinstance(c[i],str) and c[i] != 's' and c[i] != 'string':
                        seq += 1
                        if seq > 1:
                            c[i-seq+1] += ' ' + c[i]
                    
                    else:
                        seq = 0
                output2.append(c)
            for i in range(2, len(output2)):
                self.data.append([output2[i][2], output2[i][3], output2[i][-1]])
            del output2
        del outputs
        self.keys = [self.data[i][1] for i in range(len(self.data))]
    
    def _print_dataframe(self,iterations=1):
        for i in range(iterations):
            dataframe = self._write(self.ID,'',True)
            print(dataframe)
        
    
    def get(self):
        vals = self._write(self.ID,'',verbose=True)[0].split()
        readings = {k: v for k,v in zip(self.keys,vals[1:])}
        return readings
        
    def set_gas(self, gas, verbose=False):
        
        if int(self.gas_ref[gas]) != int(self.gas_ref[self.current_gas]):
            self._write(self.ID,f'G$${self.gas_ref[gas]}',verbose)
            self.current_gas = self.reverse_gas_list[int(self.gas_ref[gas])]
        
    def create_mix(self, gases: list, percentages: list):
        pass
#    
    
    def delete_mix(self, mixnum):
        pass
#    
    
    def lock(self):
        self._write(self.ID,'L')
        
    def unlock(self):
        self._write(self.ID,'U')
    
    
    def tare_press(self):
        self._write(self.ID,'PC')
    
    def tare_flow(self):
        self._write(self.ID,'V')
    
    
    def totalizer_reset(self):
        self._write(self.ID,'T')
    
    
    def change_id(self, newid: str):
        if newid.upper() != self.ID:
            val = 256 * ord(newid.upper())
            reg = int(self._write(self.ID,'R17',True).split()[-1])
            reg += val - (256 * ord(self.ID.upper()))
            self._write(self.ID,f'W17={reg}')
            self.ID = newid.upper()
    
    def change_baud(self, newbaud: int):
        #need more info on this, refdocs are hard to understand
        if newbaud != self.baud:
            idval = 256 * ord(self.ID)
            val = int(self._write(self.ID,'R17').split()[-1]) - idval
        pass
#    
    
    def change_stp(self, standardtemp, standardpress):
        if self.firmware_version == 'GP':
            raise Exception('STP modification is not available in this firmware version')
        else:
            # need to add unit checking and unit changing, also check against existing STP values
            temp = (standardtemp + 273.15) * 100000
            press = standardpress * 100000
            self._write(self.ID, f'W137={temp}')
            self._write(self.ID, f'W138={press}')
    
    def change_ntp(self, normaltemp, normalpress):
        if self.firmware_version == 'GP':
            raise Exception('NTP modification is not available in this firmware version')
        else:
            # need to add unit checking and unit changing, also check against existing NTP values
            temp = (normaltemp + 273.15) * 100000
            press = normalpress * 100000
            self._write(self.ID, f'W139={temp}')
            self._write(self.ID, f'W140={press}')
    
    
    def set_alarm(self, expression: str = ''):
        self._write(self.ID,f' ALE {expression}')
    
    def factory_restore(self):
        if self.firmware_version >= 7:
            self._write(self.ID,' FACTORY RESTORE ALL')
            time.sleep(10)
            self._flush()
        elif self.firmware_version == 'GP':
            raise Exception('Restoration of factory defualts is not available on this device.')
        else:
            self.write(self.ID,'W5683=128')
            self.write(self.ID,'Z')
            time.sleep(10)
            self._flush()
        

In [90]:
AAA = MassFlowMeter(ID='A', port='COM5', baud=19200)

In [297]:
AAA.firmware_minor

'07'

In [89]:
AAA._close()
del AAA

In [ ]:
BBB = MassFlowMeter(ID='B', port='COM5', baud=19200)

In [ ]:
BBB.firmware_minor